In [1]:
# Test on podcast generation

# from podcast.repo2txt import repo_to_text

from podcast.main import generate_podcast

# generate_podcast("This is a test input")

KeyboardInterrupt: 

In [1]:
import time
from typing import Any, Union

# Third-party imports
import requests
from bark import SAMPLE_RATE, generate_audio, preload_models
from gradio_client import Client
from openai import OpenAI
from pydantic import ValidationError
from scipy.io.wavfile import write as write_wav

# Local imports
from podcast.constants import (
    FIREWORKS_API_KEY,
    FIREWORKS_BASE_URL,
    FIREWORKS_MODEL_ID,
    FIREWORKS_MAX_TOKENS,
    FIREWORKS_TEMPERATURE,
    FIREWORKS_JSON_RETRY_ATTEMPTS,
    MELO_API_NAME,
    MELO_TTS_SPACES_ID,
    MELO_RETRY_ATTEMPTS,
    MELO_RETRY_DELAY,
    JINA_READER_URL,
    JINA_RETRY_ATTEMPTS,
    JINA_RETRY_DELAY,
)
from podcast.schema import ShortDialogue

# # Initialize clients
# fw_client = OpenAI(base_url=FIREWORKS_BASE_URL, api_key=FIREWORKS_API_KEY)
# hf_client = Client(MELO_TTS_SPACES_ID)

# # Download and load all models for Bark
# preload_models()

In [3]:
fw_client = OpenAI(base_url=FIREWORKS_BASE_URL, api_key=FIREWORKS_API_KEY)
# hf_client = Client(MELO_TTS_SPACES_ID)

In [ ]:
hf_client = Client(MELO_TTS_SPACES_ID) # this one is taking ages to load ... 


In [4]:
!pip install hume

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 23.2.1
    Uninstalling aiofiles-23.2.1:
      Successfully uninstalled aiofiles-23.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 0.8.1 requires websockets<12.0,>=10.0, but you have websockets 12.0 which is incompatible.
chainlit 1.1.402 requires aiofiles<24.0.0,>=23.1.0, but you have aiofiles 24.1.0 which is incompatible.
gradio 4.15.0 requires aiofiles<24.0,>=22.0, but you have aiofiles 24.1.0 which 

In [ ]:
def generate_script(
    system_prompt: str,
    input_text: str,
    output_model: ShortDialogue,
) -> ShortDialogue:
    """Get the dialogue from the LLM."""

    # Call the LLM
    response = call_llm(system_prompt, input_text, output_model)
    response_json = response.choices[0].message.content

    # Validate the response
    for attempt in range(FIREWORKS_JSON_RETRY_ATTEMPTS):
        try:
            first_draft_dialogue = output_model.model_validate_json(response_json)
            break
        except ValidationError as e:
            if attempt == FIREWORKS_JSON_RETRY_ATTEMPTS - 1:  # Last attempt
                raise ValueError(
                    f"Failed to parse dialogue JSON after {FIREWORKS_JSON_RETRY_ATTEMPTS} attempts: {e}"
                ) from e
            error_message = (
                f"Failed to parse dialogue JSON (attempt {attempt + 1}): {e}"
            )
            # Re-call the LLM with the error message
            system_prompt_with_error = f"{system_prompt}\n\nPlease return a VALID JSON object. This was the earlier error: {error_message}"
            response = call_llm(system_prompt_with_error, input_text, output_model)
            response_json = response.choices[0].message.content
            first_draft_dialogue = output_model.model_validate_json(response_json)

    # Call the LLM a second time to improve the dialogue
    system_prompt_with_dialogue = f"{system_prompt}\n\nHere is the first draft of the dialogue you provided:\n\n{first_draft_dialogue}."

    # Validate the response
    for attempt in range(FIREWORKS_JSON_RETRY_ATTEMPTS):
        try:
            response = call_llm(
                system_prompt_with_dialogue,
                "Please improve the dialogue. Make it more natural and engaging.",
                output_model,
            )
            final_dialogue = output_model.model_validate_json(
                response.choices[0].message.content
            )
            break
        except ValidationError as e:
            if attempt == FIREWORKS_JSON_RETRY_ATTEMPTS - 1:  # Last attempt
                raise ValueError(
                    f"Failed to improve dialogue after {FIREWORKS_JSON_RETRY_ATTEMPTS} attempts: {e}"
                ) from e
            error_message = f"Failed to improve dialogue (attempt {attempt + 1}): {e}"
            system_prompt_with_dialogue += f"\n\nPlease return a VALID JSON object. This was the earlier error: {error_message}"
    return final_dialogue


def call_llm(system_prompt: str, text: str, dialogue_format: Any) -> Any:
    """Call the LLM with the given prompt and dialogue format."""
    response = fw_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        model=FIREWORKS_MODEL_ID,
        max_tokens=FIREWORKS_MAX_TOKENS,
        temperature=FIREWORKS_TEMPERATURE,
        response_format={
            "type": "json_object",
            "schema": dialogue_format.model_json_schema(),
        },
    )
    return response

In [6]:
!pip install pyht

  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: yarl
    Found existing installation: yarl 1.9.2
    Uninstalling yarl-1.9.2:
      Successfully uninstalled yarl-1.9.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.9.3
    Uninstalling aiohttp-3.9.3:
      Successfully uninstalled aiohttp-3.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai-magics 2.7.1 requires langchain==0.0.350, but you have langchain 0.2.12 which is incompatible.
llm-swarm 0.1.0 requires datasets<3.0.0,>=2.14.6, but you have datasets 3.0.0 which is incompatible.
llm-swarm 0.1.0 requires huggingface-hub<0.21.0,>=0.20.2, but you have huggingface-hub 0.24.0 which is incompatible.
llm-swarm 0.1.0 requires pandas<3.0.

In [5]:
from pyht import Client
from dotenv import load_dotenv
from pyht.client import TTSOptions
import os
load_dotenv()

client = Client(
    user_id=os.getenv("dnjaQTim2ddvXmEgOAVbIeq8dpP2"),
    api_key=os.getenv("5809f489586045d39eec46ff5df9773e"),
)
options = TTSOptions(voice="s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json")
for chunk in client.tts("Can you tell me your account email or, ah your phone number?", options):
    # do something with the audio chunk
    print(type(chunk))

    # response.stream_to_file(speech_file_path)

ModuleNotFoundError: No module named 'pyht'

In [ ]:
from pyht import Client
from dotenv import load_dotenv
from pyht.client import TTSOptions
import os
load_dotenv()

client = Client(
    user_id=os.getenv("PLAY_HT_USER_ID"),
    api_key=os.getenv("PLAY_HT_API_KEY"),
)
options = TTSOptions(voice="s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json")
for chunk in client.tts("Can you tell me your account email or, ah your phone number?", options):
    # do something with the audio chunk
    print(type(chunk))"
    )

    response.stream_to_file(speech_file_path)

In [1]:
from podcast.utils import generate_script
from podcast.schema import ShortDialogue

ShortDialogue

KeyboardInterrupt: 

In [ ]:
generate_script("Be nice", "xxxxxxxxxxx Donald Trump is Elon Musk! hell yeah!", ShortDialogue)

PermissionDeniedError: Error code: 403 - {'error': 'unauthorized'}